# Backward Euler Method

Links:
* https://scicomp.stackexchange.com/questions/5042/how-to-implement-newtons-method-for-solving-the-algebraic-equations-in-the-back/5045#5045  -  How to implement Newton's method for solving the algebraic equations in the backward Euler method
* https://scicomp.stackexchange.com/questions/33701/understanding-butcher-tableau-when-it-comes-to-implicit-methods  -  Understanding butcher tableau when it comes to implicit methods
* https://stackoverflow.com/questions/70794806/explicit-forward-and-implicit-backward-euler-methods-in-python  -  Explicit (Forward) and Implicit (Backward) Euler Methods in Python

Take the general form of a first order differential equation (ODE).
$$
\frac{dy(t)}{dt} = f(t,y(t)) ~~ or ~~ y' = f(f,y)
$$
That has the initial condition $y(t_0) = y_0$
Discretizing the derivative $\frac{dy(t)}{dt}$ to numerically approximate the ODE with a backwards stepping its approximation is as follows
$$
\frac{dy(t)}{dt} = \frac{y_n - y_{n-1}}{\Delta t}
$$
Substituting the backwards step derivative approximation into the ODE yields the following
$$
\frac{y_n - y_{n-1}}{\Delta t} = f(t_n,y_n)
$$
The equation can be re-written to
$$
y_n= y_{n-1} + \Delta t f(t_n,y_n)
$$
Then, by shifting the equation, in positive step direction, and substituting $\Delta t$ with $h$, the Backwards Euler formula is derived
$$
y_{n+1} = y_{n} + hf(t_{n+1},y_{n+1})
$$